In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.stats import norm
import math

In [2]:
rates = pd.read_csv('./InterestRates.csv')
#rates['key'] = rates['pricedate'].str.cat(rates['maturity'] , sep =", ")
rates['pricedate_str'] = rates['pricedate']
rates['pricedate'] = pd.to_datetime(rates.pricedate, format='%Y-%m-%d', errors='ignore')
#print (rates.dtypes)
#rates.info()
rates[['number1','month']] = rates.maturity.str.split("M",expand=True,)
rates[['number2','year']] = rates.maturity.str.split("Y",expand=True,)

#So, I want to place number with pfv where pfv is not null, how can I achieve that?
rates['number'] = np.where(~rates['year'].isnull(),rates['number2'],rates['number1'])
rates['number'] = pd.to_numeric(rates['number'])
rates['MorY'] = np.where(~rates['year'].isnull(),'Y','M')

rates = rates.drop(columns=['number1','number2','month','year'])
rates['months'] = np.where(rates['MorY']=='Y',rates['number']*12,rates['number']*1)

#generating key2
#total_rows['ColumnID'] = total_rows['ColumnID'].astype(str)
rates['months_str'] = rates['months'].astype(str)
rates['key2'] = rates['pricedate_str'].str.cat(rates['months_str'], sep=',')

rates.info()
rates.head()

#for the risk free rates I am thinking on using the smallest one, 1 month
#rates_1m = rates[rates['maturity']=='1M']
#rates_1m.info()
#rates_1m.head()

#len(rates.key.unique())
#len(rates.maturity.unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7085 entries, 0 to 7084
Data columns (total 9 columns):
pricedate        7085 non-null datetime64[ns]
maturity         7085 non-null object
bidrate          7085 non-null float64
pricedate_str    7085 non-null object
number           7085 non-null int64
MorY             7085 non-null object
months           7085 non-null int64
months_str       7085 non-null object
key2             7085 non-null object
dtypes: datetime64[ns](1), float64(1), int64(2), object(5)
memory usage: 498.2+ KB


,pricedate,maturity,bidrate,pricedate_str,number,MorY,months,months_str,key2
0,2020-03-02,100M,1.463333,2020-03-02,100,M,100,100,"2020-03-02,100"
1,2020-03-02,101M,1.466667,2020-03-02,101,M,101,101,"2020-03-02,101"
2,2020-03-02,102M,1.470001,2020-03-02,102,M,102,102,"2020-03-02,102"
3,2020-03-02,103M,1.478335,2020-03-02,103,M,103,103,"2020-03-02,103"
4,2020-03-02,104M,1.486669,2020-03-02,104,M,104,104,"2020-03-02,104"


In [3]:
prices = pd.read_csv('./PricesFile1.csv')
prices['pricedate_str'] = prices['pricedate']
prices['delivdate_str'] = prices['delivdate']
prices['key'] = prices['pricedate_str'].str.cat(prices['delivdate_str'], sep=',')

#drop other variables
prices = prices[prices['priceindex']=='WTI NYMEX LIGHT SWEET']

#fixing dates
#pricedate and delivdate
prices['pricedate'] = pd.to_datetime(prices.pricedate, format='%Y-%m-%d', errors='ignore')
prices['delivdate'] = pd.to_datetime(prices.delivdate, format='%Y-%m-%d', errors='ignore')
#prices.info()
#negative prices
neg_prices = prices[prices['price']<0]


#calculating duration ****QUESTION Should we drop negatives according to scott answer?
prices['duration'] = prices['delivdate'] - prices['pricedate']
prices['duration_months'] = round(prices['duration'].dt.days/30).astype(int)

#generating key2
prices['duration_months'][prices['duration_months']==0] = 1
prices['duration_months_str'] = prices['duration_months'].astype(str)
prices['key2'] = prices['pricedate_str'].str.cat(prices['duration_months_str'], sep=',')

#prices.info()

#drop negative duration  **63 elements dropped
prices_drop = prices[prices['duration'].dt.days<0]
prices = prices[prices['duration'].dt.days>=0]

prices = prices.drop(columns=['pricedate_str','delivdate_str'])

prices[(prices['pricedate']=='2020-04-20')& (prices['delivdate']=='2020-05-01')]

#Day of negative
#prices[prices['pricedate']=='2020-04-20']

#Checking if prices are unique for same dates
#len(prices.key.unique())

#prices.priceindex.unique()
#neg_prices.info()
#neg_prices.head()
#prices.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,priceindex,pricedate,delivdate,price,key,duration,duration_months,duration_months_str,key2
5208,WTI NYMEX LIGHT SWEET,2020-04-20,2020-05-01,-37.631,"2020-04-20,2020-05-01",11 days,1,1,"2020-04-20,1"


In [4]:
wti_imvl20=pd.read_csv("./WTI_ImpliedVols2020.csv")
wti_imvl21=pd.read_csv("./WTI_ImpliedVols2021.csv")
wti_imvl22=pd.read_csv("./WTI_ImpliedVols2022.csv")

#Append
frames=[wti_imvl20,wti_imvl21,wti_imvl22]
wti_imvl = pd.concat(frames)

#generating key
wti_imvl['volatilitydate_str'] = wti_imvl['volatilitydate']
wti_imvl['begtime_str'] = wti_imvl['begtime']
wti_imvl['key'] = wti_imvl['volatilitydate_str'].str.cat(wti_imvl['begtime_str'], sep=',')

#fixing dates
#volatilitydate and begtime
wti_imvl['volatilitydate'] = pd.to_datetime(wti_imvl.volatilitydate, format='%Y-%m-%d', errors='ignore')
wti_imvl['begtime'] = pd.to_datetime(wti_imvl.begtime, format='%Y-%m-%d', errors='ignore')

#calculating duration ****QUESTION Should we drop negatives according to scott answer?
wti_imvl['duration'] = wti_imvl['begtime'] - wti_imvl['volatilitydate']
wti_imvl['duration_months'] = round(wti_imvl['duration'].dt.days/30,0).astype(int)

#Take previous day volatility

#the negative day
neg_day = wti_imvl[(wti_imvl['volatilitydate']=='2020-04-20') & (wti_imvl['begtime']=='2020-05-01')]


#drop negative duration  **No elements dropped
wti_imvl = wti_imvl[wti_imvl['duration_months']>=0]
wti_imvl = wti_imvl[wti_imvl['duration_months']>=0]
#Checking if wti_imvl are unique for same dates
len(wti_imvl.key.unique())
#wti_imvl.info()

###generating key2
#wti_imvl['duration_months'][wti_imvl['duration_months']==0] = 1
#wti_imvl['duration_months_str'] = wti_imvl['duration_months'].astype(str)
#wti_imvl['key2'] = wti_imvl['volatilitydate_str'].str.cat(wti_imvl['duration_months_str'], sep=',')

#####Volatility
#key                1198 non-null object
#strikeprice        1198 non-null float64
#volatility         1198 non-null float64
#volatilityindex    1198 non-null object
wti_imvl = wti_imvl.drop(columns=['volatilitydate','begtime','duration','duration_months','volatilitydate_str','begtime_str'])

#Left join
master_wti = pd.merge(prices,wti_imvl,on='key',how='left')

#Replace NA's by the previous observation 
na_master_wti = master_wti[np.isnan(master_wti['volatility'])]
na_master_wti.reset_index(drop=True, inplace=True)

for i in range(len(na_master_wti)):
    first_date = na_master_wti['pricedate'][i]
    last_date = na_master_wti['delivdate'][i]
    sub_set = master_wti[(master_wti['pricedate']<=first_date)& (master_wti['delivdate']==last_date)].sort_values(by='pricedate', ascending=False)
    sub_set.reset_index(drop=True, inplace=True)
    j=0
    while np.isnan(sub_set['volatility'][j]) and sub_set['pricedate'][j] > datetime.strptime('2020-03-02', '%Y-%m-%d'):
        j = j+1    # update counter
    
    if j>0 and sub_set['pricedate'][j] > datetime.strptime('2020-03-02', '%Y-%m-%d'):
        index_save = master_wti.index[(master_wti['pricedate']==first_date)& (master_wti['delivdate']==last_date)][0]
        master_wti['volatilityindex'].at[index_save]=(sub_set['volatilityindex'][j+1])
        master_wti['strikeprice'].at[index_save]=(sub_set['strikeprice'][j+1])
        master_wti['volatility'].at[index_save]=(sub_set['volatility'][j+1])
        #master_wti['key2'].at[index_save]=(sub_set['key2'][j+1])
        del(index_save)

    del(first_date,last_date,sub_set)

master_wti = master_wti.dropna()

#master_wti[(master_wti['pricedate']=='2020-04-20')& (master_wti['delivdate']=='2020-05-01')]
            
#len(master_wti[np.isnan(master_wti['volatility'])])

master_wti.info()
#master_wti.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 77794 entries, 0 to 78541
Data columns (total 12 columns):
priceindex             77794 non-null object
pricedate              77794 non-null datetime64[ns]
delivdate              77794 non-null datetime64[ns]
price                  77794 non-null float64
key                    77794 non-null object
duration               77794 non-null timedelta64[ns]
duration_months        77794 non-null int64
duration_months_str    77794 non-null object
key2                   77794 non-null object
volatilityindex        77794 non-null object
strikeprice            77794 non-null float64
volatility             77794 non-null float64
dtypes: datetime64[ns](2), float64(3), int64(1), object(5), timedelta64[ns](1)
memory usage: 7.7+ MB


In [5]:
#Merging Rates
rates2 = rates.drop(columns=['pricedate','pricedate_str','number','MorY','months','months_str'])

#Left join
master_wti = pd.merge(master_wti,rates2,on='key2',how='left')
master_wti[(master_wti['pricedate']=='2020-04-20')& (master_wti['delivdate']=='2020-05-01')]

master_wti.to_csv ('master_wti_complete.csv', index = False, header=True)
master_wti.info()
#master_wti.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77794 entries, 0 to 77793
Data columns (total 14 columns):
priceindex             77794 non-null object
pricedate              77794 non-null datetime64[ns]
delivdate              77794 non-null datetime64[ns]
price                  77794 non-null float64
key                    77794 non-null object
duration               77794 non-null timedelta64[ns]
duration_months        77794 non-null int64
duration_months_str    77794 non-null object
key2                   77794 non-null object
volatilityindex        77794 non-null object
strikeprice            77794 non-null float64
volatility             77794 non-null float64
maturity               77794 non-null object
bidrate                77794 non-null float64
dtypes: datetime64[ns](2), float64(4), int64(1), object(6), timedelta64[ns](1)
memory usage: 8.9+ MB


In [6]:
#Black 76


# t = pricedate          1198 non-null datetime64[ns]
# T = delivdate          1198 non-null datetime64[ns]
# F(t,T) = price              1198 non-null float64
# T - t = duration_n         1198 non-null int64
# K = strikeprice        1198 non-null float64
# sigma = volatility         1198 non-null float64
# r = bidrate            1198 non-null float64

master_wti.rename(columns={'pricedate': 't', 'delivdate': 'T', 'price': 'F', 'strikeprice': 'K', 'volatility': 'sigma', 'bidrate': 'r'}, inplace=True)

master_wti['T_t'] = master_wti['duration'].dt.days

#FIX A Te
#For creating the loop, check if Te_t < T, if true, calculate the call value
#Te_t = 15/360

master_wti = master_wti[master_wti['T_t']>30]
Te_t = (master_wti['T_t']-30)/360

#Calculating call value
Nd1 = norm.cdf((np.log(master_wti['F']/master_wti['K']) + (1/2 * master_wti['sigma']**2 * Te_t))/(master_wti['sigma'] * np.sqrt(Te_t)))
Nd2 = norm.cdf((np.log(master_wti['F']/master_wti['K']) - (1/2 * master_wti['sigma']**2 * Te_t))/(master_wti['sigma'] * np.sqrt(Te_t)))

master_wti['call'] = np.exp(-(master_wti['r']/100)*Te_t)*(master_wti['F']*Nd1 - master_wti['K']*Nd2)

Nd1_put = norm.cdf(-(np.log(master_wti['F']/master_wti['K']) + (1/2 * master_wti['sigma']**2 * Te_t))/(master_wti['sigma'] * np.sqrt(Te_t)))
Nd2_put = norm.cdf(-(np.log(master_wti['F']/master_wti['K']) - (1/2 * master_wti['sigma']**2 * Te_t))/(master_wti['sigma'] * np.sqrt(Te_t)))

master_wti['put'] = np.exp(-(master_wti['r']/100)*Te_t)*(master_wti['K']*Nd2_put - master_wti['F']*Nd1_put )

#master_wti[(master_wti['t']=='2020-04-20')& (master_wti['T']=='2020-05-01')]
master_wti.info()
#master_wti.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75361 entries, 81 to 77793
Data columns (total 17 columns):
priceindex             75361 non-null object
t                      75361 non-null datetime64[ns]
T                      75361 non-null datetime64[ns]
F                      75361 non-null float64
key                    75361 non-null object
duration               75361 non-null timedelta64[ns]
duration_months        75361 non-null int64
duration_months_str    75361 non-null object
key2                   75361 non-null object
volatilityindex        75361 non-null object
K                      75361 non-null float64
sigma                  75361 non-null float64
maturity               75361 non-null object
r                      75361 non-null float64
T_t                    75361 non-null int64
call                   75361 non-null float64
put                    75361 non-null float64
dtypes: datetime64[ns](2), float64(6), int64(2), object(6), timedelta64[ns](1)
memory usage: 10.

In [8]:
master_wti.to_csv ('master_wti_call_put_black76.csv', index = False, header=True)
master_wti.head()

,priceindex,t,T,F,key,duration,duration_months,duration_months_str,key2,volatilityindex,K,sigma,maturity,r,T_t,call,put
81,WTI NYMEX LIGHT SWEET,2020-03-02,2020-05-01,46.919,"2020-03-02,2020-05-01",60 days,2,2,"2020-03-02,2",V_WTI NYMEX LIGHT SWEET,25.0,0.767,2M,0.995003,60,21.905789,0.004956
82,WTI NYMEX LIGHT SWEET,2020-03-02,2020-05-01,46.919,"2020-03-02,2020-05-01",60 days,2,2,"2020-03-02,2",V_WTI NYMEX LIGHT SWEET,25.5,0.757,2M,0.995003,60,21.407156,0.005909
83,WTI NYMEX LIGHT SWEET,2020-03-02,2020-05-01,46.919,"2020-03-02,2020-05-01",60 days,2,2,"2020-03-02,2",V_WTI NYMEX LIGHT SWEET,26.0,0.746,2M,0.995003,60,20.908585,0.006923
84,WTI NYMEX LIGHT SWEET,2020-03-02,2020-05-01,46.919,"2020-03-02,2020-05-01",60 days,2,2,"2020-03-02,2",V_WTI NYMEX LIGHT SWEET,26.5,0.736,2M,0.995003,60,20.410268,0.008192
85,WTI NYMEX LIGHT SWEET,2020-03-02,2020-05-01,46.919,"2020-03-02,2020-05-01",60 days,2,2,"2020-03-02,2",V_WTI NYMEX LIGHT SWEET,27.0,0.726,2M,0.995003,60,19.912149,0.009658


In [ ]:
## For date specific Plot
#'2020-04-20'
master_wti_1=master_wti[master_wti.t =='2020-04-20']
#df_wti_1=df_wti_1[df_wti_1.begtime =='2020-11-01']
master_wti_1.set_index(master_wti_1.t, inplace=True)
master_wti_1.drop(columns="t", inplace=True)
#master_wti_1


plt.rcParams['figure.figsize'] = (25,10)
plt.plot(master_wti_1['T'],master_wti_1['call'])
plt.xticks(rotation=90)
plt.title('Call prices starting April 20, 2020')

In [ ]:
## For date specific Plot
plt.rcParams['figure.figsize'] = (25,10)
plt.plot(master_wti_1['T'],master_wti_1['put'])
plt.xticks(rotation=90)
plt.title('Put prices starting April 20, 2020')